In [1]:
import logging
from naslib.defaults.trainer import Trainer
from naslib.optimizers import DARTSOptimizer
from naslib.search_spaces import DartsSearchSpace
from naslib.utils import utils, setup_logger, get_config_from_args, set_seed, log_args
from naslib.search_spaces.core.graph import Graph, EdgeData
from naslib.search_spaces.core import primitives as ops
from torch import nn
from fvcore.common.config import CfgNode
from copy import deepcopy
from IPython.display import clear_output

device: cuda:0
device: cpu
device: cuda:0
device: cuda:0
device: cuda:0
device: cuda:0


In [8]:
config = utils.get_config_from_args(config_type='nas')
# config.search.epochs = 1 # for testing
config.optimizer = 'darts'
utils.set_seed(config.seed)
clear_output(wait=True)
utils.log_args(config)

logger = setup_logger(config.save + '/log.log')
logger.setLevel(logging.INFO)

[06/27 10:13:28 nl.utils.utils]: dataset....................................cifar10
[06/27 10:13:28 nl.utils.utils]: seed.............................................0
[06/27 10:13:28 nl.utils.utils]: search_space...........................nasbench201
[06/27 10:13:28 nl.utils.utils]: out_dir........................................run
[06/27 10:13:28 nl.utils.utils]: optimizer....................................darts
[06/27 10:13:28 nl.utils.utils]: searchacq_fn_optimization: random_sampling
acq_fn_type: its
arch_learning_rate: 0.0003
arch_weight_decay: 0.001
batch_size: 256
checkpoint_freq: 1000
cutout: False
cutout_length: 16
cutout_prob: 1.0
data_size: 25000
debug_predictor: False
drop_path_prob: 0.0
encoding_type: adjacency_one_hot
epochs: 100
fidelity: -1
gpu: None
grad_clip: 5
k: 10
learning_rate: 0.025
learning_rate_min: 0.001
max_mutations: 1
momentum: 0.9
num_arches_to_mutate: 2
num_candidates: 20
num_ensemble: 3
num_init: 10
output_weights: True
population_size: 30
predictor_t

In [9]:
from naslib.search_spaces.core.graph import Graph, EdgeData
from naslib.search_spaces.core import primitives as ops
from torch import nn
from copy import deepcopy

class DartsSearchSpace(Graph):

    OPTIMIZER_SCOPE = [
        'a_stage_1',
        'a_stage_2', 
        'a_stage_3'
    ]

    QUERYABLE = False

    def __init__(self):
        super().__init__()

        channels = [(16 * 5 * 5, 120), (120, 84), (84, 10)]
        stages = ['a_stage_1', 'a_stage_2', 'a_stage_3']

        # cell definition
        activation_cell = Graph()
        activation_cell.name = 'activation_cell'
        activation_cell.add_node(1) # input node
        activation_cell.add_node(2) # intermediate node
        activation_cell.add_node(3) # output node
        activation_cell.add_edges_from([(1, 2, EdgeData())]) # mutable intermediate edge
        activation_cell.edges[1, 2].set('cell_name', 'activation_cell') 
        activation_cell.add_edges_from([(2, 3, EdgeData().finalize())]) # immutable output edge

        # macroarchitecture definition
        self.name = 'makrograph'
        self.add_node(1) # input node
        self.add_node(2) # intermediate node
        for i, scope in zip(range(3, 6), stages):
            self.add_node(i, subgraph=deepcopy(activation_cell).set_scope(scope).set_input([i-1])) # activation node i
            self.nodes[i]['subgraph'].name = scope # set 
        self.add_node(6) # output node
        self.add_edges_from([(i, i+1, EdgeData()) for i in range(1, 6)])
        self.edges[1, 2].set('op',
            ops.Sequential(
                nn.Conv2d(3, 6, 5),
                nn.MaxPool2d(2),
                nn.Conv2d(6, 16, 5),
                nn.MaxPool2d(2),
                nn.Flatten()
            )) # convolutional edge
        
        for scope, (in_dim, out_dim) in zip(stages, channels):
            self.update_edges(
                update_func=lambda edge: self._set_ops(edge, in_dim, out_dim),
                scope=scope,
                private_edge_data=True,
            )

    def _set_ops(self, edge, in_dim, out_dim):
        if out_dim != 10:
            edge.data.set('op', [
                ops.Sequential(nn.Linear(in_dim, out_dim), nn.ReLU()),
                ops.Sequential(nn.Linear(in_dim, out_dim), nn.Hardswish()),
                ops.Sequential(nn.Linear(in_dim, out_dim), nn.LeakyReLU()),
                ops.Sequential(nn.Linear(in_dim, out_dim), nn.Identity())
            ])
        else:
            edge.data.set('op', [
                ops.Sequential(nn.Linear(in_dim, out_dim), nn.Softmax(dim=1))
            ])  

In [10]:
search_space = DartsSearchSpace()

In [11]:
optimizer = DARTSOptimizer(config)
optimizer.adapt_search_space(search_space)

[06/27 10:13:37 nl.optimizers.oneshot.darts.optimizer]: Parsed graph:
Graph a_stage_1:
 Graph(
  (a_stage_1-edge(1,2)): MixedOp(
    (primitive-0): Sequential(
      (op): Sequential(
        (0): Linear(in_features=400, out_features=120, bias=True)
        (1): ReLU()
      )
    )
    (primitive-1): Sequential(
      (op): Sequential(
        (0): Linear(in_features=400, out_features=120, bias=True)
        (1): Hardswish()
      )
    )
    (primitive-2): Sequential(
      (op): Sequential(
        (0): Linear(in_features=400, out_features=120, bias=True)
        (1): LeakyReLU(negative_slope=0.01)
      )
    )
    (primitive-3): Sequential(
      (op): Sequential(
        (0): Linear(in_features=400, out_features=120, bias=True)
        (1): Identity()
      )
    )
  )
  (a_stage_1-edge(2,3)): Identity()
)
Graph a_stage_2:
 Graph(
  (a_stage_2-edge(1,2)): MixedOp(
    (primitive-0): Sequential(
      (op): Sequential(
        (0): Linear(in_features=120, out_features=84, bias=Tru

In [12]:
trainer = Trainer(optimizer, config)
trainer.search()

[06/27 10:13:38 nl.defaults.trainer]: param size = 0.236858MB
[06/27 10:13:38 nl.defaults.trainer]: Start training
Files already downloaded and verified
Files already downloaded and verified
[06/27 10:13:39 nl.optimizers.oneshot.darts.optimizer]: Arch weights (alphas, last column argmax): 
-0.000529, +0.001313, +0.000432, +0.000184, 1
+0.000824, -0.001669, +0.000387, +0.000960, 3
+0.001344, 0
[06/27 10:13:40 nl.defaults.trainer]: Epoch 0-0, Train loss: 2.30296, validation loss: 2.30231, learning rate: [0.025]
[06/27 10:13:45 nl.defaults.trainer]: Epoch 0-30, Train loss: 2.30282, validation loss: 2.30232, learning rate: [0.025]
[06/27 10:13:50 nl.defaults.trainer]: Epoch 0-60, Train loss: 2.30240, validation loss: 2.30287, learning rate: [0.025]
[06/27 10:13:55 nl.defaults.trainer]: Epoch 0-90, Train loss: 2.30253, validation loss: 2.30225, learning rate: [0.025]
[06/27 10:14:00 nl.defaults.trainer]: Epoch 0-120, Train loss: 2.30295, validation loss: 2.30237, learning rate: [0.025]
[06/

[06/27 10:15:58 nl.optimizers.oneshot.darts.optimizer]: Arch weights before discretization: [Parameter containing:
tensor([-0.3235, -0.3281, -0.3506,  0.4281], device='cuda:0',
       requires_grad=True), Parameter containing:
tensor([-0.2555, -0.2628, -0.2551,  0.3309], device='cuda:0',
       requires_grad=True), Parameter containing:
tensor([1.1911e-43], device='cuda:0', requires_grad=True)]
[06/27 10:15:58 nl.defaults.trainer]: Epoch 5 done. Train accuracy (top1, top5): 25.34000, 72.82000, Validation accuracy: 26.10059, 72.66195
[06/27 10:15:58 nl.optimizers.oneshot.darts.optimizer]: Arch weights (alphas, last column argmax): 
-0.323537, -0.328100, -0.350614, +0.428123, 3
-0.255544, -0.262764, -0.255105, +0.330919, 3
+0.000000, 0
[06/27 10:16:02 nl.defaults.trainer]: Epoch 6-20, Train loss: 2.18517, validation loss: 2.14826, learning rate: [0.02478744700874427]
[06/27 10:16:07 nl.defaults.trainer]: Epoch 6-50, Train loss: 2.20430, validation loss: 2.20526, learning rate: [0.0247874

[06/27 10:18:10 nl.defaults.trainer]: Epoch 11-88, Train loss: 2.12945, validation loss: 2.11816, learning rate: [0.02429056922745071]
[06/27 10:18:15 nl.defaults.trainer]: Epoch 11-118, Train loss: 2.11600, validation loss: 2.10283, learning rate: [0.02429056922745071]
[06/27 10:18:18 nl.optimizers.oneshot.darts.optimizer]: Arch weights before discretization: [Parameter containing:
tensor([-0.2908, -0.2723, -0.3173,  0.4096], device='cuda:0',
       requires_grad=True), Parameter containing:
tensor([-0.2758, -0.2630, -0.2219,  0.3516], device='cuda:0',
       requires_grad=True), Parameter containing:
tensor([1.1911e-43], device='cuda:0', requires_grad=True)]
[06/27 10:18:18 nl.defaults.trainer]: Epoch 11 done. Train accuracy (top1, top5): 35.25429, 81.03429, Validation accuracy: 34.34934, 80.16081
[06/27 10:18:18 nl.optimizers.oneshot.darts.optimizer]: Arch weights (alphas, last column argmax): 
-0.290776, -0.272266, -0.317259, +0.409648, 3
-0.275819, -0.262987, -0.221904, +0.351612,

[06/27 10:20:17 nl.defaults.trainer]: Epoch 17-11, Train loss: 2.08191, validation loss: 2.02331, learning rate: [0.023328904324047328]
[06/27 10:20:22 nl.defaults.trainer]: Epoch 17-42, Train loss: 2.06835, validation loss: 2.09555, learning rate: [0.023328904324047328]
[06/27 10:20:27 nl.defaults.trainer]: Epoch 17-72, Train loss: 2.03738, validation loss: 2.03639, learning rate: [0.023328904324047328]
[06/27 10:20:32 nl.defaults.trainer]: Epoch 17-102, Train loss: 2.05480, validation loss: 2.05313, learning rate: [0.023328904324047328]
[06/27 10:20:37 nl.defaults.trainer]: Epoch 17-132, Train loss: 2.04586, validation loss: 2.03606, learning rate: [0.023328904324047328]
[06/27 10:20:38 nl.optimizers.oneshot.darts.optimizer]: Arch weights before discretization: [Parameter containing:
tensor([-0.2299, -0.2131, -0.2790,  0.3594], device='cuda:0',
       requires_grad=True), Parameter containing:
tensor([-0.2752, -0.2143, -0.2151,  0.3447], device='cuda:0',
       requires_grad=True), P

[06/27 10:22:33 nl.optimizers.oneshot.darts.optimizer]: Arch weights (alphas, last column argmax): 
-0.191816, -0.181581, -0.246612, +0.325780, 3
-0.265507, -0.215335, -0.239729, +0.360228, 3
+0.000000, 0
[06/27 10:22:34 nl.defaults.trainer]: Epoch 23-2, Train loss: 2.03155, validation loss: 2.00680, learning rate: [0.022001332835565518]
[06/27 10:22:39 nl.defaults.trainer]: Epoch 23-32, Train loss: 2.00301, validation loss: 2.04757, learning rate: [0.022001332835565518]
[06/27 10:22:44 nl.defaults.trainer]: Epoch 23-62, Train loss: 2.02952, validation loss: 2.03021, learning rate: [0.022001332835565518]
[06/27 10:22:49 nl.defaults.trainer]: Epoch 23-92, Train loss: 1.97464, validation loss: 2.05167, learning rate: [0.022001332835565518]
[06/27 10:22:54 nl.defaults.trainer]: Epoch 23-122, Train loss: 2.04171, validation loss: 2.03158, learning rate: [0.022001332835565518]
[06/27 10:22:56 nl.optimizers.oneshot.darts.optimizer]: Arch weights before discretization: [Parameter containing:


...

[06/27 10:43:04 nl.defaults.trainer]: Epoch 75 done. Train accuracy (top1, top5): 60.62571, 94.40857, Validation accuracy: 58.67359, 94.06364
[06/27 10:43:04 nl.optimizers.oneshot.darts.optimizer]: Arch weights (alphas, last column argmax): 
+0.111382, +0.086710, -0.062312, +0.021563, 0
-0.255282, -0.177951, -0.312404, +0.421237, 3
+0.000000, 0
[06/27 10:43:04 nl.defaults.trainer]: Epoch 76-2, Train loss: 1.87498, validation loss: 1.86836, learning rate: [0.00425237647094306]
[06/27 10:43:09 nl.defaults.trainer]: Epoch 76-32, Train loss: 1.80951, validation loss: 1.92093, learning rate: [0.00425237647094306]
[06/27 10:43:14 nl.defaults.trainer]: Epoch 76-62, Train loss: 1.86040, validation loss: 1.88246, learning rate: [0.00425237647094306]
[06/27 10:43:19 nl.defaults.trainer]: Epoch 76-92, Train loss: 1.87222, validation loss: 1.91990, learning rate: [0.00425237647094306]
[06/27 10:43:24 nl.defaults.trainer]: Epoch 76-122, Train loss: 1.85886, validation loss: 1.85686, learning rate: 

[06/27 10:45:22 nl.optimizers.oneshot.darts.optimizer]: Arch weights before discretization: [Parameter containing:
tensor([ 0.1223,  0.1131, -0.0561, -0.0004], device='cuda:0',
       requires_grad=True), Parameter containing:
tensor([-0.2620, -0.1715, -0.3374,  0.4341], device='cuda:0',
       requires_grad=True), Parameter containing:
tensor([1.1911e-43], device='cuda:0', requires_grad=True)]
[06/27 10:45:22 nl.defaults.trainer]: Epoch 81 done. Train accuracy (top1, top5): 61.30571, 94.39714, Validation accuracy: 59.12409, 94.06079
[06/27 10:45:22 nl.optimizers.oneshot.darts.optimizer]: Arch weights (alphas, last column argmax): 
+0.122254, +0.113092, -0.056064, -0.000362, 0
-0.262021, -0.171534, -0.337417, +0.434138, 3
+0.000000, 0
[06/27 10:45:26 nl.defaults.trainer]: Epoch 82-20, Train loss: 1.86792, validation loss: 1.88708, learning rate: [0.002868064893975819]
[06/27 10:45:31 nl.defaults.trainer]: Epoch 82-50, Train loss: 1.86205, validation loss: 1.87226, learning rate: [0.002

[06/27 10:47:34 nl.defaults.trainer]: Epoch 87-83, Train loss: 1.82481, validation loss: 1.89604, learning rate: [0.0019869444917922276]
[06/27 10:47:40 nl.defaults.trainer]: Epoch 87-113, Train loss: 1.84941, validation loss: 1.84690, learning rate: [0.0019869444917922276]
[06/27 10:47:43 nl.optimizers.oneshot.darts.optimizer]: Arch weights before discretization: [Parameter containing:
tensor([ 0.1461,  0.1363, -0.0527, -0.0271], device='cuda:0',
       requires_grad=True), Parameter containing:
tensor([-0.2853, -0.1909, -0.3706,  0.4672], device='cuda:0',
       requires_grad=True), Parameter containing:
tensor([1.1911e-43], device='cuda:0', requires_grad=True)]
[06/27 10:47:43 nl.defaults.trainer]: Epoch 87 done. Train accuracy (top1, top5): 61.47143, 94.50857, Validation accuracy: 60.23038, 94.46567
[06/27 10:47:43 nl.optimizers.oneshot.darts.optimizer]: Arch weights (alphas, last column argmax): 
+0.146070, +0.136280, -0.052718, -0.027112, 0
-0.285336, -0.190907, -0.370565, +0.467

[06/27 10:49:41 nl.defaults.trainer]: Epoch 93-7, Train loss: 1.80829, validation loss: 1.90000, learning rate: [0.0012889988567350316]
[06/27 10:49:46 nl.defaults.trainer]: Epoch 93-37, Train loss: 1.88105, validation loss: 1.85761, learning rate: [0.0012889988567350316]
[06/27 10:49:51 nl.defaults.trainer]: Epoch 93-67, Train loss: 1.85404, validation loss: 1.83059, learning rate: [0.0012889988567350316]
[06/27 10:49:56 nl.defaults.trainer]: Epoch 93-97, Train loss: 1.87670, validation loss: 1.85115, learning rate: [0.0012889988567350316]
[06/27 10:50:01 nl.defaults.trainer]: Epoch 93-127, Train loss: 1.84609, validation loss: 1.84077, learning rate: [0.0012889988567350316]
[06/27 10:50:03 nl.optimizers.oneshot.darts.optimizer]: Arch weights before discretization: [Parameter containing:
tensor([ 0.1611,  0.1403, -0.0495, -0.0400], device='cuda:0',
       requires_grad=True), Parameter containing:
tensor([-0.2886, -0.2017, -0.3964,  0.4878], device='cuda:0',
       requires_grad=True)

[06/27 10:51:59 nl.defaults.trainer]: Epoch 98 done. Train accuracy (top1, top5): 62.53714, 94.43143, Validation accuracy: 60.39291, 94.36017
[06/27 10:51:59 nl.optimizers.oneshot.darts.optimizer]: Arch weights (alphas, last column argmax): 
+0.164252, +0.156129, -0.061027, -0.046950, 0
-0.286570, -0.203149, -0.408523, +0.497054, 3
+0.000000, 0
[06/27 10:52:03 nl.defaults.trainer]: Epoch 99-28, Train loss: 1.80052, validation loss: 1.86617, learning rate: [0.0010059212756112208]
[06/27 10:52:08 nl.defaults.trainer]: Epoch 99-58, Train loss: 1.79381, validation loss: 1.88038, learning rate: [0.0010059212756112208]
[06/27 10:52:14 nl.defaults.trainer]: Epoch 99-88, Train loss: 1.81332, validation loss: 1.83958, learning rate: [0.0010059212756112208]
[06/27 10:52:19 nl.defaults.trainer]: Epoch 99-118, Train loss: 1.88037, validation loss: 1.82860, learning rate: [0.0010059212756112208]
[06/27 10:52:22 nl.optimizers.oneshot.darts.optimizer]: Arch weights before discretization: [Parameter c

In [13]:
trainer.evaluate_oneshot()

[06/27 10:58:44 nl.defaults.trainer]: Start one-shot evaluation
Files already downloaded and verified
Files already downloaded and verified
[06/27 10:58:50 nl.defaults.trainer]: Evaluation finished


60.559999985758466